In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import re
import time
tf.__version__

'1.0.0'

In [6]:
vocab_lines = open(r'InsuranceQnA-master\vocabulary.txt', encoding='utf-8', errors="ignore").read().split('\n')
question_lines = open(r'InsuranceQnA-master\InsuranceQAquestionanslabelraw.encoded', encoding='utf-8', errors="ignore").read().split('\n')
answer_lines = open(r'InsuranceQnA-master\InsuranceQAlabel2answerraw.encoded', encoding='utf-8', errors="ignore").read().split('\n')

In [7]:
id2line = {}
for line in vocab_lines:
    _line = line.split('\t')
    if len(_line) == 2:
        id2line[_line[0]] = _line[1]

In [8]:
convs, ansid = [], []
for line in question_lines[:-1]:
    _line = line.split('\t')
    ansid.append(_line[2].split(' '))
    convs.append(_line[1])

In [9]:
question_lines[0].split('\t')

['medicare-insurance',
 'idx_1285 idx_1010 idx_467 idx_47610 idx_18488 idx_65760',
 '16696']

In [10]:
convs1 = [ ]
for line in answer_lines[:-1]:
    _line = line.split('\t')
    convs1.append(_line[1])

In [11]:
print(convs[:2])
print(ansid[:2])
print(convs1[:2])

['idx_1285 idx_1010 idx_467 idx_47610 idx_18488 idx_65760', 'idx_3815 idx_604 idx_605 idx_891 idx_136 idx_5293 idx_65761']
[['16696'], ['10277']]
['idx_1 idx_2 idx_3 idx_4 idx_5 idx_6 idx_7 idx_8 idx_9 idx_10 idx_11 idx_12 idx_13 idx_14 idx_3 idx_12 idx_15 idx_16 idx_17 idx_8 idx_18 idx_19 idx_20 idx_21 idx_3 idx_12 idx_14 idx_22 idx_20 idx_23 idx_24 idx_25 idx_26 idx_27 idx_28 idx_29 idx_8 idx_30 idx_19 idx_11 idx_4 idx_31 idx_32 idx_22 idx_33 idx_34 idx_35 idx_36 idx_37 idx_30 idx_38 idx_39 idx_11 idx_40 idx_41 idx_42 idx_43 idx_44 idx_22 idx_45 idx_46 idx_11 idx_47 idx_48 idx_49 idx_18 idx_50 idx_20 idx_44 idx_22 idx_51 idx_14 idx_52 idx_53 idx_22 idx_40 idx_21 idx_3 idx_54 idx_46 idx_11 idx_55 idx_56 idx_57 idx_58 idx_59 idx_60 idx_61 idx_62 idx_8 idx_50 idx_11 idx_45 idx_63 idx_64 idx_3 idx_65 idx_66 idx_3 idx_67 idx_68 idx_69 idx_70 idx_14 idx_3 idx_71 idx_72 idx_73 idx_21 idx_74 idx_5 idx_75 idx_76 idx_12 idx_8 idx_77 idx_78 idx_15 idx_79 idx_49 idx_18 idx_19 idx_11 idx_54 idx_4

In [13]:
count=0
questions, answers = [], []
for a in range(len(ansid)):
      for b in range(len(ansid[a])):
            questions.append(convs[a])
            count+=1
for a in range(len(ansid)):
      for b in range(len(ansid[a])):
            answers.append(convs1[int(ansid[a][b])-1])

In [14]:
### sample code for above task in my version
# ques=[]
# for i in questions:
#     a=[]
#     que_tokens=i.split(' ')
#     for token in que_tokens:
#         word=id2line[token.strip()]
#         a.append(word)
#     ques.append(' '.join(a))

In [15]:
ques, ans  =[], []
m=0
while m<len(questions):
       i=0
       a=[]
       while i < (len(questions[m].split(' '))):
            a.append(id2line[questions[m].split(' ')[i]])
            i=i+1
       ques.append(' '.join(a))
       m=m+1

In [16]:
n=0
while n<len(answers):  
        j=0
        b=[]
        while j < (len(answers[n].split(' '))):
            b.append(id2line[answers[n].split(' ')[j]])
            j=j+1
        ans.append(' '.join(b))
        n=n+1     

In [17]:
def clean_text(text):
        """Cleaning the text by replacing the abbreviated words with their proper full replacement, and converting all the characters to lower case"""

        text = text.lower()

        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "that is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"n'", "ng", text)
        text = re.sub(r"'bout", "about", text)
        text = re.sub(r"'til", "until", text)
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)

        return text

# Applying the 'clean_text()' function on the set of Questions and Answers
clean_questions = []
for question in ques:
    clean_questions.append(clean_text(question))

clean_answers = []    
for answer in ans:
    clean_answers.append(clean_text(answer))

In [18]:
min_line_length, max_line_length = 2, 100
short_questions_temp, short_answers_temp = [], []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

In [19]:
short_questions, short_answers = [], []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [20]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Including <PAD> token in sentence to make all batches of same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [21]:
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [22]:
threshold = 1
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [23]:
questions_vocab_to_int = {}
word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1

answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [24]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1

for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [58]:
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [25]:
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)

In [26]:
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [27]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

unk_ratio = round(unk_count/word_count,4)*100

In [28]:
sorted_questions = []
short_questions1 = []
sorted_answers = []
short_answers1= []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            short_questions1.append(short_questions[i[0]])
            sorted_answers.append(answers_int[i[0]])
            short_answers1.append(short_answers[i[0]])

for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print(short_questions1[i])
    print(short_answers1[i])
    print()

[4021, 17664]
[4021, 17664, 5205, 2432, 17027, 14142, 6270, 4276, 15811, 4021, 17664, 3681, 1378, 12300, 1646, 4021, 17664, 3681, 1378, 18628, 11454, 18948, 13460, 487, 7648, 10699, 7742, 18074, 18171, 9669, 12817, 6907, 14940, 11454, 18338, 11391, 480, 14065, 8954, 10420, 9669, 5162, 14940, 6060, 18948, 12817, 6907, 7742, 12817, 17027, 4021, 17664, 12817, 6907, 4021, 6202]
why can
why can a simple question but yet so complex why can someone do this or why can someone do that i have often pondered for hours to come up with the answer and i believe after years of thoughtprovoking consultation with friends and relativesi have the answer to the question why can the answer why not

[14553, 4165, 5833]
[17329, 16346, 7874, 18478, 17143, 5833, 68, 18070, 9583, 3071, 14090, 964, 7648, 15716, 5833, 10132, 14940, 12817, 12935, 14065, 5515, 15716, 5629, 12284, 15716, 5833, 9583, 5057, 5629, 14940, 18074, 7932, 12817, 18801, 14065, 1327, 4294, 14142, 4543, 13497, 11308, 223, 10561, 5515, 12981, 1

In [31]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    lr = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    return input_data, targets, lr, keep_prob

def process_encoding_input(target_data, vocab_to_int, batch_size):

    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell, cell_bw = enc_cell, sequence_length = sequence_length, inputs = rnn_inputs, dtype=tf.float32)
    return enc_state

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.keras.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    train_decoder_fn = tf.keras.seq2seq.attention_decoder_fn_train(encoder_state[0], att_keys, att_vals,  att_score_fn, att_construct_fn,  name = "attn_dec_train")

    train_pred, _, _ = tf.keras.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder_fn,  dec_embed_input, sequence_length, scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)

    return output_fn(train_pred_drop)

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], att_keys, att_vals,  att_score_fn, att_construct_fn,  name = "attn_dec_train")

    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder_fn,  dec_embed_input, sequence_length, scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)

    return output_fn(train_pred_drop)

def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, encoder_state[0],  att_keys, att_vals,  att_score_fn, att_construct_fn,
                        dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, name = "attn_dec_inf")

    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)

    return infer_logits

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):

    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)

        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None,  scope=decoding_scope, weights_initializer = weights, biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, dec_cell,  dec_embed_input, sequence_length,  decoding_scope, output_fn, keep_prob, batch_size)

        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, vocab_to_int['<GO>'], vocab_to_int['<EOS>'],
                    sequence_length - 1, vocab_size,  decoding_scope, output_fn, keep_prob, batch_size)

    return train_logits, infer_logits

In [32]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size,
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers,
                  questions_vocab_to_int):

    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, answers_vocab_size+1,  enc_embedding_size, initializer = tf.random_uniform_initializer(0,1))

    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    train_logits, infer_logits = decoding_layer(dec_embed_input, dec_embeddings, enc_state, questions_vocab_size,
                            sequence_length, rnn_size, num_layers, questions_vocab_to_int,  keep_prob, batch_size)

    return train_logits, infer_logits

In [33]:
epochs = 50
batch_size = 64
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [34]:
tf.reset_default_graph()
# Starting the session
sess = tf.InteractiveSession()

In [35]:
input_data, targets, lr, keep_prob = model_inputs()
sequence_length = tf.placeholder_with_default(max_line_length, None, name="sequence_length")

In [36]:
input_shape = tf.shape(input_data)

train_logits, inference_logits = seq2seq_model( tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int),
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers,  questions_vocab_to_int)

In [37]:
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Calculating Loss function
    cost = tf.contrib.seq2seq.sequence_loss( train_logits, targets, tf.ones([input_shape[0], sequence_length]))

    # Using Adam Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Performing Gradient Clipping to handle the vanishing gradient problem
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [38]:
def batch_data(questions, answers, batch_size):

    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield pad_questions_batch, pad_answers_batch

In [39]:
train_valid_split = int(len(sorted_questions)*0.15)

train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))

16242
2866


In [40]:
display_step = 20        # Check training loss after every 20 batches

stop_early = 0

stop = 5                 # If the validation loss decreases after 5 consecutive checks, stop training

validation_check = ((len(train_questions))//batch_size//2)-1        # Counter for checking validation loss

total_train_loss = 0     # Record the training loss for each display step

summary_valid_loss = []     # Record the validation loss for saving improvements in the model

checkpoint= "./best_model.ckpt"   # creating the checkpoint file in the current directory

sess.run(tf.global_variables_initializer())

In [41]:
for epoch_i in range(1, epochs+1):
    for batch_i, (questions_batch, answers_batch) in enumerate(
            batch_data(train_questions, train_answers, batch_size)):
        start_time = time.time()
        _, loss = sess.run(
            [train_op, cost],
            {input_data: questions_batch, targets: answers_batch,  lr: learning_rate,
             sequence_length: answers_batch.shape[1], keep_prob: keep_probability})

        total_train_loss += loss
        end_time = time.time()
        batch_time = end_time - start_time

        if batch_i % display_step == 0:
            print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                  .format(epoch_i, epochs, batch_i,
                          len(train_questions) // batch_size, total_train_loss / display_step,
                          batch_time*display_step))
            total_train_loss = 0

        if batch_i % validation_check == 0 and batch_i > 0:
            total_valid_loss = 0
            start_time = time.time()
            for batch_ii, (questions_batch, answers_batch) in enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                valid_loss = sess.run(
                cost, {input_data: questions_batch, targets: answers_batch, lr: learning_rate,
                       sequence_length: answers_batch.shape[1], keep_prob: 1})
                total_valid_loss += valid_loss
            end_time = time.time()
            batch_time = end_time - start_time
            avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
            print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))

            # Reduce learning rate, but not below its minimum value
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            summary_valid_loss.append(avg_valid_loss)
            if avg_valid_loss <= min(summary_valid_loss):
                print('New Record!')
                stop_early = 0
                saver = tf.train.Saver()
                saver.save(sess, checkpoint)

            else:
                print("No Improvement.")
                stop_early += 1
                if stop_early == stop:
                    break

    if stop_early == stop:
        print("Stopping Training.")
        break

Epoch   1/50 Batch    0/253 - Loss:  0.498, Seconds: 364.91
Epoch   1/50 Batch   20/253 - Loss:  8.311, Seconds: 361.21
Epoch   1/50 Batch   40/253 - Loss:  4.567, Seconds: 380.52
Epoch   1/50 Batch   60/253 - Loss:  4.404, Seconds: 478.50
Epoch   1/50 Batch   80/253 - Loss:  4.480, Seconds: 456.56
Epoch   1/50 Batch  100/253 - Loss:  4.371, Seconds: 383.58
Epoch   1/50 Batch  120/253 - Loss:  4.324, Seconds: 307.75
Valid Loss:  4.130, Seconds: 228.40
New Record!
Epoch   1/50 Batch  140/253 - Loss:  4.292, Seconds: 385.96
Epoch   1/50 Batch  160/253 - Loss:  4.271, Seconds: 384.78
Epoch   1/50 Batch  180/253 - Loss:  4.201, Seconds: 379.80
Epoch   1/50 Batch  200/253 - Loss:  4.118, Seconds: 386.72
Epoch   1/50 Batch  220/253 - Loss:  4.046, Seconds: 389.18
Epoch   1/50 Batch  240/253 - Loss:  4.015, Seconds: 386.88
Valid Loss:  3.935, Seconds: 273.58
New Record!
Epoch   2/50 Batch    0/253 - Loss:  2.580, Seconds: 348.58
Epoch   2/50 Batch   20/253 - Loss:  3.965, Seconds: 377.47
Epoc

KeyboardInterrupt: 

In [69]:
def question_to_seq(question, vocab_to_int):
    """Creating the question to be taken as input by the model"""
    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in question.split()]

In [90]:
random = np.random.choice(len(short_questions))
input_question = short_questions[random]
print(input_question)

how long does a home insurance claim stay on your record


In [91]:
input_question = question_to_seq(input_question, questions_vocab_to_int)
input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
batch_shell = np.zeros((batch_size, max_line_length))
batch_shell[0] = input_question

In [93]:
answer_logits = sess.run(inference_logits, {input_data: batch_shell, keep_prob: 1.0})[0]

In [85]:
pad_q = questions_vocab_to_int["<PAD>"]
pad_a = answers_vocab_to_int["<PAD>"]

In [94]:
print('Question')
print('Word Ids: {}'.format([i for i in input_question if i != pad_q]))
print('Input Words: {}'.format([questions_int_to_vocab[i] for i in input_question if i != pad_q]))
print('\n')

Question
Word Ids: [3016, 4712, 5354, 5205, 2506, 5515, 44, 11179, 17338, 4619, 16337]
Input Words: ['how', 'long', 'does', 'a', 'home', 'insurance', 'claim', 'stay', 'on', 'your', 'record']




In [95]:
print('\nAnswer')
print('Word Ids: {}'.format([i for i in np.argmax(answer_logits, 1) if i != pad_a]))

print('Response Words: {}'.format([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a]))

print('\n')


Answer
Word Ids: [12817, 6907, 11308, 5205, 3610, 17027, 18628, 11308, 5205, 3610, 17027, 18628, 11034, 17664, 3264, 10176, 5515, 7648, 5205, 3610, 17027, 18628, 11034, 17664, 3264, 5205, 10176, 5515, 11263, 7648, 12817, 11263, 7742, 12190, 11399, 7742, 12817, 11263, 7742, 12190, 12817, 11263, 7742, 12190, 12817, 11263, 7742, 12190, 12817, 11263, 7742, 12190, 12817, 11263, 7742, 12190, 12817, 11263, 7742, 12190, 11399, 7742, 12817, 11263, 17329, 11034, 18948, 5205, 44, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 12190, 2974, 7742, 18948, 5205, 44]
Response Words: ['the', 'answer', 'is', 'a', 'great', 'question', 'that', 'is', 'a', 'great', 'question', 'that', 'you', 'can', 'get', 'life', 'insurance', 'for', 'a', 'great', 'question', 'that', 'you', 'can', 'get', 'a', 'life', 'insurance', 'policy', 'for', 'the', 'policy', 'to', 'be', 'paid', 'to', 'the', 'policy', 'to', 'be'

In [88]:
print(' '.join(([questions_int_to_vocab[i] for i in input_question if i != pad_q])))

print(' '.join(([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a])))

can life insurance be a tax deduction
yes you can get a great question that you can get life insurance for a great question that you can get a life insurance policy for the policy to be paid to the policy to be paid to the policy to be the policy to be paid to the policy to be paid to the policy to be paid to the policy if you have a claim to be able to be able to be able to be able to be able to be able to be able to be able to be able to have a claim to be
